In [1]:
import pandas as pd
import cv2
import numpy as np

In [2]:
folder='/notebooks/udacity/car_data/data/'

In [3]:
def load(folder):
    print (folder)
    df=pd.read_csv(folder+'driving_log.csv',\
                names=['center','left','right','steering','trottle','brake','speed'],skiprows=1)

    correction=0.2
    images=np.empty((len(df)*6,160,320,3), dtype=np.uint8)
    target=np.empty((len(df))*6, dtype=np.float)

    #center
    for i,fname in enumerate(df['center'].values):
       # print (folder+'IMG/'+fname.split('/')[-1])
        images[i]=cv2.imread(folder+'IMG/'+fname.split('/')[-1])

    #left
    for i,fname in enumerate(df['left'].values):
        images[i+len(df)]=cv2.imread(folder+'IMG/'+fname.split('/')[-1])

    #right
    for i,fname in enumerate(df['right'].values):
        images[i+2*len(df)]=cv2.imread(folder+'IMG/'+fname.split('/')[-1])

    target[0:len(df)]=df.steering.values
    target[len(df):2*len(df)]=df.steering.values+correction
    target[2*len(df):3*len(df)]=df.steering.values-correction

    target[3*len(df):6*len(df)]=-target[0:3*len(df)]

    for i in range(len(df)*3):
        images[i+3*len(df)]=np.fliplr(images[i])
        
    return images, target


In [4]:
datadirs=['/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/']

In [7]:
data= [load(a ) for a in datadirs]

In [10]:
images=np.concatenate([a[0] for a in data])
target=np.concatenate([a[1] for a in data])

del data

In [11]:
from keras.models import Sequential
from keras.layers import Flatten,Dense, Convolution2D, Activation, \
MaxPooling2D, Lambda,Conv2D,Dropout, Cropping2D,\
AveragePooling2D

Using TensorFlow backend.


In [12]:
model=Sequential()

model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5))

#model.add(Convolution2D(20, 5, 5, border_mode="same",
#input_shape=(160,320,3)))
model.add(AveragePooling2D(pool_size=(4,4)))
model.add(Conv2D(32, 3, 3,
                 activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))

In [13]:
model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])

In [14]:
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("model_cnn.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=1, mode='min')

In [ ]:
model.fit(images, target, validation_split=0.2, shuffle=True, nb_epoch=10,batch_size=128,\
             callbacks=[checkpoint, early_stop])

Train on 76680 samples, validate on 19170 samples
Epoch 1/10
76680/76680 [==============================] - 65s - loss: 0.0382 - mean_squared_error: 0.0382 - val_loss: 0.1142 - val_mean_squared_error: 0.1142
Epoch 2/10
76680/76680 [==============================] - 63s - loss: 0.0246 - mean_squared_error: 0.0246 - val_loss: 0.1109 - val_mean_squared_error: 0.1109
Epoch 3/10
76680/76680 [==============================] - 63s - loss: 0.0240 - mean_squared_error: 0.0240 - val_loss: 0.1082 - val_mean_squared_error: 0.1082
Epoch 4/10
76680/76680 [==============================] - 64s - loss: 0.0236 - mean_squared_error: 0.0236 - val_loss: 0.1103 - val_mean_squared_error: 0.1103
Epoch 5/10
76680/76680 [==============================] - 63s - loss: 0.0233 - mean_squared_error: 0.0233 - val_loss: 0.1076 - val_mean_squared_error: 0.1076
Epoch 6/10
76680/76680 [==============================] - 63s - loss: 0.0232 - mean_squared_error: 0.0232 - val_loss: 0.1064 - val_mean_squared_error: 0.1064
Ep

In [11]:
model.save_weights('weights')